In [3]:
import io
from IPython.nbformat import current

def execute_notebook(nbfile):
    
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    
    ip = get_ipython()
    
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
        
execute_notebook("common.ipynb")

Tue Jun 12 07:17:39 2018


In [2]:
epochs = 100 
spikeMiliseconds = 100
spikeInterval = spikeMiliseconds * ms
spikeIntervalUnformatted = spikeMiliseconds * .001
dictionaryLongitude = 4
spikesPerChar=3
firstLayerSize = 15

dictionary = dictionary()
spiketimes = dictionary.spikeTimes(dictionaryLongitude, spikeInterval, spikesPerChar, epochs)
LIK = SpikeGeneratorGroup(firstLayerSize, spiketimes)

N = 1000
taum = 300 * ms
Vt = -54 * mV
Vr = -60 * mV
El = -74 * mV
taue = 100 * ms
taui = 100 * ms

eqs = Equations('''
	  dV/dt  = (-V+ge-gi)/taum : volt
	  dge/dt = -ge/taue        : volt
	  dgi/dt = -gi/taui        : volt
	  ''')
ADDS = NeuronGroup(N=4, model=eqs,threshold=Vt, reset=Vr)

excitatory = Connection(LIK, ADDS , 'ge',structure='dense',max_delay=30 * ms,
                        delay=lambda i, j:(10 + 20 * rand()) * ms )
Wexcitatory = np.random.uniform(0,15,[15,4]) * mV
excitatory.connect(LIK,ADDS,Wexcitatory)
Ap = 1 * mV
Am = 1 * mV
stdpE=ExponentialSTDP(excitatory,taue,taum,Ap,Am,wmax=15 * mV,interactions='all',update='additive')

inhibitory = Connection(ADDS, ADDS , 'gi',delay=0*ms,structure='dense')
#Connect adds layer via lateral inhibitory connections
#the diagonal should be 0 to not auto-inhibate
Winhibitory = np.random.uniform(0,200,[4,4]) * mV
diagonal = np.diag_indices(Winhibitory.shape[0])
Winhibitory[diagonal] = 0;

inhibitory.connect(ADDS,ADDS,Winhibitory)
stdpI=ExponentialSTDP(inhibitory,taue,taum,Ap,Am,wmax=4000 * mV,interactions='all',update='additive')

spikeMonitor = SpikeMonitor(ADDS)
Mv  = StateMonitor(ADDS, 'V', record=True)
Mge = StateMonitor(ADDS, 'ge', record=True)
Mgi = StateMonitor(ADDS, 'gi', record=True)


#save weights!
weights_excitatory = []
weights_inhibitory = []
@network_operation
def record_weights():
    weights_excitatory.append(array(excitatory.W))
    weights_inhibitory.append(array(inhibitory.W))

run(spikesPerChar * dictionaryLongitude * spikeInterval * epochs , report='text')

TypeError: data type "c1" not understood

We will now save all the important variables : neuron voltages, neuron spikes, and weights to a file called simulation.hdf5
while saving the data, we will generate downsampled arrays, so when we plot we don't use that much RAM.
if you are on ubuntu you can use:

    hdfview simulation.hdf5
to explore the dataset.
we haven't implemented a method for downsampling sparse arrays yet, so spiketime isn't save , but it shouldn't be to hard to do that

In [ ]:
bp = brianPlotter('simulation.hdf5')

bp.saveData('spikes', np.asarray(spikeMonitor.spikes), downsample=False);

Mv =  np.transpose(Mv[:])
Mge =  np.transpose(Mge[:])
Mgi =  np.transpose(Mgi[:])
voltage = np.concatenate((Mv[...,np.newaxis], Mge[...,np.newaxis]), axis=2)
voltage = np.concatenate((voltage, Mgi[...,np.newaxis]), axis=2)
bp.saveData('voltage',voltage)

bp.saveData('excitatory', np.asarray(weights_excitatory))
bp.saveData('inhibitory', np.asarray(weights_inhibitory))

del bp #you have to delete the object so the file is closed

If you want to see the analysis of this simulation click [here](analysis.ipynb).

Remember to shutdown this notebook to release memory